## Import Library

In [1]:
import json  # For working with JSON data
import numpy as np  # For numerical operations
import pandas as pd  # For data manipulation and analysis
from tqdm.notebook import tqdm  # For displaying progress bars in Jupyter notebooks

# Open and load the JSON file containing contact data
with open('contacts.json') as f:
    data = json.load(f)

## Replace Empty String with NaN

In [2]:
df = pd.DataFrame(data)
df = df.replace('', np.NaN)

df

,Id,Email,Phone,Contacts,OrderId
0,0,gkzAbIy@qq.com,NaN,1,NaN
1,1,NaN,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov
2,2,NaN,9125983679,0,NaN
3,3,mdllpYmE@gmail.com,NaN,0,bHquEnCbbsGLqllwryxPsNOxa
4,4,NaN,300364407,2,NaN
...,...,...,...,...,...
499995,499995,NaN,10072395382,2,whJlUOGNqjxCRzmIRdURQvlNv
499996,499996,NaN,571709661031,4,JqIXOONvrwvJxZqNxCYHqnrKS
499997,499997,NaN,4541459979,2,beXCZSzcHaBwAYoDcpQqjuAFO
499998,499998,RzSDsyH@hotmail.com,98947185431,1,ehjeFACGiwrERQxbziMxwOWku


## Grouping Tickets

In [ ]:
%%time  # Measure the execution time of this cell (specific to Jupyter notebooks)

email_group = df.groupby('Email').Id.agg(lambda x: set(x))  # Group by email and collect associated IDs
phone_group = df.groupby('Phone').Id.agg(lambda x: set(x))  # Group by phone and collect associated IDs
order_group = df.groupby('OrderId').Id.agg(lambda x: set(x))  # Group by order ID and collect associated IDs

d = {i: set() for i in df.Id}

# Populate the dictionary by merging sets of IDs from the email, phone, and order groups
for ids in email_group:
    for id in ids:
        d[id] |= set(ids)  # Union the sets of IDs for each email group
for ids in phone_group:
    for id in ids:
        d[id] |= set(ids)  # Union the sets of IDs for each phone group
for ids in order_group:
    for id in ids:
        d[id] |= set(ids)  # Union the sets of IDs for each order group

# Merge across groups again to ensure all connected IDs are grouped together
for i in tqdm(range(3)):  # Use tqdm to show progress for the 3 iterations
    for id, ids in d.items():
        for id_ in list(ids):
            d[id] |= d[id_]

In [4]:
# Create a dictionary mapping each 'Id'
id_to_contact = df.set_index('Id').Contacts.to_dict()

# Define a function to calculate the sum of 'Contacts' for a given set of IDs
def get_sum_contact(ids_set):
    return sum([id_to_contact[id] for id in ids_set])

In [5]:
df['set'] = df.Id.apply(lambda x: d[x])  # Use the dictionary 'd' to look up the set of connected IDs
df['trace'] = df.set.apply(lambda x: '-'.join(map(str, sorted(list(x)))))  # Convert the set to a sorted list, then to a string
df['n_con'] = df.set.apply(lambda x: str(get_sum_contact(x)))  # Use the 'get_sum_contact' function to calculate the sum
df['out'] = df.trace + ', ' + df.n_con  # Concatenate the two columns with a comma separator

out = df[['Id', 'out']]
out.columns = ['ticket_id', 'ticket_trace/contact']

out

,ticket_id,ticket_trace/contact
0,0,"0, 1"
1,1,"1-2458-98519-115061-140081-165605-476346, 12"
2,2,"2-159312-322639-348955, 4"
3,3,"3, 0"
4,4,"4, 2"
...,...,...
499995,499995,"499995, 2"
499996,499996,"499996, 4"
499997,499997,"499997, 2"
499998,499998,"121111-499998, 5"


In [6]:
out.to_csv('output.csv', index=False)